# MantisShrimp

> Built on top of [pytorch-lightining](https://github.com/PyTorchLightning/pytorch-lightning), `MantisShrimp` is an object detection framework focused on application.

MantisShrimp is in very early development, all contributions are welcome!

Be sure to check the `issues` board =)

## Install

```python
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip install mantisshrimp
```

## How to use

Check the `examples` folder for a ton of usage cases. It should be very easy to get started.